In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
import re
import datetime
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth
from Project.Database import Db

In [47]:
year1_hourly, meta = Db.load_data(hourly=True, meta=True, year=1)

In [48]:
status_columns = meta.loc[(meta['Parameter'] == "Status_OnOff")]["Unnamed: 0"].tolist()
year1_status_columns = year1_hourly[status_columns].copy()

In [49]:
for i in status_columns:
    year1_status_columns.loc[year1_status_columns[i] < 0, i] = 0
    year1_status_columns.loc[year1_status_columns[i] > 0, i] = 1
year1_status_columns = year1_status_columns.fillna(0)

In [50]:
associations = apriori(year1_status_columns, min_support=0.2, use_colnames=True, verbose=3)
associations

Processing 20 combinations | Sampling itemset size 1086
       support                                           itemsets
0     0.426093                (DHW_StatusSolenoidColdKitchenSink)
1     0.289807                  (DHW_StatusSolenoidColdMBAShower)
2     0.426093                 (DHW_StatusSolenoidHotKitchenSink)
3     0.289807                   (DHW_StatusSolenoidHotMBAShower)
4     0.223719                             (Load_StatusBR2Lights)
...        ...                                                ...
2589  0.230225  (Load_StatusKitchenLightsA, Load_StatusLRLight...
2590  0.230225  (Load_StatusSensHeatPrntBDOWN, Load_StatusLRLi...
2591  0.230225  (Load_StatusKitchenLightsA, Load_StatusLRLight...
2592  0.230225  (Load_StatusKitchenLightsA, Load_StatusLRLight...
2593  0.230225  (Load_StatusKitchenLightsA, Load_StatusLRLight...

[2594 rows x 2 columns]


In [51]:
df_ar = association_rules(associations, metric="confidence", min_threshold=0.9999)
df_ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(DHW_StatusSolenoidColdKitchenSink),(DHW_StatusSolenoidHotKitchenSink),0.426093,0.426093,0.426093,1.0,2.346906,0.244538,inf
1,(DHW_StatusSolenoidHotKitchenSink),(DHW_StatusSolenoidColdKitchenSink),0.426093,0.426093,0.426093,1.0,2.346906,0.244538,inf
2,(DHW_StatusSolenoidColdMBAShower),(DHW_StatusSolenoidHotMBAShower),0.289807,0.289807,0.289807,1.0,3.450571,0.205819,inf
3,(DHW_StatusSolenoidHotMBAShower),(DHW_StatusSolenoidColdMBAShower),0.289807,0.289807,0.289807,1.0,3.450571,0.205819,inf
4,(Load_StatusKitchenLightsB),(Load_StatusKitchenLightsA),0.389453,0.389453,0.389453,1.0,2.567702,0.237779,inf
...,...,...,...,...,...,...,...,...,...
12634,"(Load_StatusSensHeatPrntBDOWN, DHW_StatusSolen...","(Load_StatusLRLights3, DHW_StatusSolenoidColdK...",0.230225,0.260473,0.230225,1.0,3.839176,0.170258,inf
12635,"(Load_StatusSensHeatPrntBDOWN, Load_StatusLRLi...","(Load_StatusKitchenLightsA, DHW_StatusSolenoid...",0.230225,0.260473,0.230225,1.0,3.839176,0.170258,inf
12636,"(Load_StatusSensHeatPrntBDOWN, Load_StatusLRLi...","(Load_StatusKitchenLightsA, DHW_StatusSolenoid...",0.230225,0.260473,0.230225,1.0,3.839176,0.170258,inf
12637,"(Load_StatusSensHeatPrntBDOWN, DHW_StatusSolen...","(Load_StatusKitchenLightsA, Load_StatusLRLight...",0.230225,0.260473,0.230225,1.0,3.839176,0.170258,inf


# With Ronnis feature selection

In [52]:
year1_minutes, meta = Db.load_data(hourly=False, meta=True, year=1)

In [53]:
#Extract appliances from the meta data
appliance_condition = (lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        (~self["Unnamed: 0"].str.contains("Child")) &
        (~self["Unnamed: 0"].str.contains("Prnt")) &
        (~self["Unnamed: 0"].str.contains("StatusLatentload"))
))

appliances_statusOnOff = meta.loc[appliance_condition]["Unnamed: 0"].tolist()

appliance_location = meta.loc[appliance_condition][["Unnamed: 0", "Measurement_Location"]].sort_values(
    by="Measurement_Location", ascending=True).rename(columns={"Unnamed: 0": "Appliance"}).set_index("Appliance")

appliance_location

,Measurement_Location
Appliance,
Load_StatusBA1Lights,Bath1
Load_StatusBA2Lights,Bath2
Load_StatusBR2Lights,Bedroom2
Load_StatusPlugLoadBR2Laptop,Bedroom2
Load_StatusBR3Lights,Bedroom3
Load_StatusPlugLoadBR3Laptop,Bedroom3
Load_StatusBR4Lights,Bedroom4
Load_StatusPlugLoadDesktopPCMonitor,Bedroom4
Load_StatusPlugLoadVacuum,Bedroom4


In [54]:
#Combine minute data with the extracted appliances into a dataframe
minute_appliances_status = year1_minutes[appliance_location.index.tolist()].copy()

In [57]:
#Replaces values to be binary 0 or 1.
for i in appliances_statusOnOff:
    minute_appliances_status.loc[minute_appliances_status[i] < 0, i] = 0
    minute_appliances_status.loc[minute_appliances_status[i] > 0, i] = 1
minute_appliances_status = minute_appliances_status.fillna(0)
minute_appliances_status

,Load_StatusBA1Lights,Load_StatusBA2Lights,Load_StatusBR2Lights,Load_StatusPlugLoadBR2Laptop,Load_StatusBR3Lights,Load_StatusPlugLoadBR3Laptop,Load_StatusBR4Lights,Load_StatusPlugLoadDesktopPCMonitor,Load_StatusPlugLoadVacuum,Load_StatusDRLights,...,Load_StatusLRLights1,Load_StatusMBALights,Load_StatusPlugLoadHairDryerCurlIron,Load_StatusPlugLoadFan,Load_StatusPlugLoadHeatingPad,Load_StatusPlugLoadIron,Load_StatusMBRLights1,Load_StatusPlugLoadMBRBlueRay,Load_StatusPlugLoadMBRTV,Load_StatusMBRLights2
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
518788,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
518789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
518790,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [74]:
associations = fpgrowth(minute_appliances_status, min_support=0.2, use_colnames=True, verbose=3)
associations

5 itemset(s) from tree conditioned on items ()
0 itemset(s) from tree conditioned on items (Load_StatusKitchenLightsB)
1 itemset(s) from tree conditioned on items (Load_StatusKitchenLightsA)
3 itemset(s) from tree conditioned on items (Load_StatusKitchenLightsC)
0 itemset(s) from tree conditioned on items (Load_StatusPlugLoadLRTV)
0 itemset(s) from tree conditioned on items (Load_StatusPlugLoadDesktopPCMonitor)


,support,itemsets
0,0.247344,(Load_StatusKitchenLightsB)
1,0.247301,(Load_StatusKitchenLightsA)
2,0.247299,(Load_StatusKitchenLightsC)
3,0.217388,(Load_StatusPlugLoadLRTV)
4,0.229144,(Load_StatusPlugLoadDesktopPCMonitor)
5,0.247301,"(Load_StatusKitchenLightsB, Load_StatusKitchen..."
6,0.247299,"(Load_StatusKitchenLightsC, Load_StatusKitchen..."
7,0.247299,"(Load_StatusKitchenLightsC, Load_StatusKitchen..."
8,0.247299,"(Load_StatusKitchenLightsC, Load_StatusKitchen..."


In [75]:
df_ar = association_rules(associations, metric="confidence", min_threshold=0.5)
df_ar

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(Load_StatusKitchenLightsB),(Load_StatusKitchenLightsA),0.247344,0.247301,0.247301,0.999829,4.042955,0.186133,4.390286e+03
1,(Load_StatusKitchenLightsA),(Load_StatusKitchenLightsB),0.247301,0.247344,0.247301,1.000000,4.042955,0.186133,inf
2,(Load_StatusKitchenLightsC),(Load_StatusKitchenLightsA),0.247299,0.247301,0.247299,1.000000,4.043648,0.186142,inf
3,(Load_StatusKitchenLightsA),(Load_StatusKitchenLightsC),0.247301,0.247299,0.247299,0.999992,4.043648,0.186142,9.656997e+04
4,(Load_StatusKitchenLightsC),(Load_StatusKitchenLightsB),0.247299,0.247344,0.247299,1.000000,4.042955,0.186131,inf
5,(Load_StatusKitchenLightsB),(Load_StatusKitchenLightsC),0.247344,0.247299,0.247299,0.999821,4.042955,0.186131,4.199414e+03
6,"(Load_StatusKitchenLightsC, Load_StatusKitchen...",(Load_StatusKitchenLightsA),0.247299,0.247301,0.247299,1.000000,4.043648,0.186142,inf
7,"(Load_StatusKitchenLightsC, Load_StatusKitchen...",(Load_StatusKitchenLightsB),0.247299,0.247344,0.247299,1.000000,4.042955,0.186131,inf
8,"(Load_StatusKitchenLightsB, Load_StatusKitchen...",(Load_StatusKitchenLightsC),0.247301,0.247299,0.247299,0.999992,4.043648,0.186142,9.656997e+04
9,(Load_StatusKitchenLightsC),"(Load_StatusKitchenLightsB, Load_StatusKitchen...",0.247299,0.247301,0.247299,1.000000,4.043648,0.186142,inf
